In [51]:
from pathlib import Path
import sys

project_root = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / 'Source' / 'ai').exists()), None)
if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain.tools import Tool
from langgraph.graph import END, StateGraph
import operator
import requests
from Source.ai.Multi_Agent.Source.Main.Tools import poem_tools, sentiment_tools, travel_tools, weather_tools, planner_tools
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_1 import Coordinator_Agent_1, Flight_Agent_1, Hotel_Agent_1, Travel_Agent_1
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_2 import Coordinator_Agent_2, Flight_Agent_2, Hotel_Agent_2, Travel_Agent_2
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_3 import Coordinator_Agent_3, Flight_Agent_3, Hotel_Agent_3, Travel_Agent_3
from typing import TypedDict, Annotated, List, Any, Dict, Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from Source.ai.Multi_Agent.Source.Main.Memory.memory.memory import MemoryManager, memory_manager
from Source.ai.Multi_Agent.Source.Main.Memory.memory import long_term_memory
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import uuid
import os

In [52]:
def new_session(user_id: str = "default_user", clear_history: bool = True, keep_preferences: bool = True, auto_continue: bool = False, replay_last_n: int = 20) -> str:
    sid = memory_manager.start_new_session(user_id=user_id, clear_history=clear_history, keep_preferences=keep_preferences)
    print(f"New session started: {sid}")
    if auto_continue:
        initial_state = build_state_from_memory(user_id=user_id, max_messages=replay_last_n)
        run_langgraph_chat(initial_state=initial_state)
    return sid

sid = new_session()

New session started: session_20250928_154914


In [53]:
load_dotenv()

llm = ChatOllama(model="llama3:8b")

# Sử dụng singleton memory_manager dùng chung cho các tool
current_session_id = memory_manager.get_session_id("default_user")
print(f"Session ID: {current_session_id}")

class AgentState(TypedDict):
    input: str
    messages: Annotated[List[str], operator.add]
    
prompt = hub.pull("hwchase17/react")

Session ID: session_20250928_154914


In [ ]:

weatherllm_agent = create_react_agent(llm, [weather_tools.weatherllm_tool], prompt) 
weatherllm_agent_executor = AgentExecutor(agent=weatherllm_agent, tools=[weather_tools.weatherllm_tool], verbose=True, handle_parsing_errors=True, max_iterations=3, early_stopping_method="force") 

def call_weatherllm_agent(state: AgentState):
    result = weatherllm_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
    return {"messages": [f"Thời tiết tại: {result['output']}"]}

sentiment_agent = create_react_agent(llm, [sentiment_tools.sentiment_tool], prompt) 
sentiment_agent_executor = AgentExecutor(agent=sentiment_agent, tools=[sentiment_tools.sentiment_tool], verbose=True, handle_parsing_errors=True, max_iterations=3, early_stopping_method="force")

def call_sentiment_agent(state: AgentState):
    result = sentiment_agent_executor.invoke({"input": f"Phân tích cảm xúc của đoạn text sau: {state['input']}"})
    return {"messages": [f"Phân tích cảm xúc: {result['output']}"]} 

poet_agent = create_react_agent(llm, [poem_tools.poem_tool], prompt) 
poet_agent_executor = AgentExecutor(agent=poet_agent, tools=[poem_tools.poem_tool], verbose=True, handle_parsing_errors=True, max_iterations=3, early_stopping_method="force") 
def call_poet_agent(state: AgentState):
    result = poet_agent_executor.invoke({"input": f"Hãy viết một bài thơ về: {state['input']}"})
    return {"messages": [f"Bài thơ: {result['output']}"]}

travel_agent = create_react_agent(llm, [travel_tools.travel_tool], prompt) 
travel_agent_executor = AgentExecutor(agent=travel_agent, tools=[travel_tools.travel_tool], verbose=True, handle_parsing_errors=True, max_iterations=3, early_stopping_method="force") 
def call_travel_agent(state: AgentState):
    result = travel_agent_executor.invoke({"input": f"Phân tích lịch trình du lịch: {state['input']}"})
    return {"messages": [f"Phân tích lịch trình du lịch: {result['output']}"]}

# Planner agent
planner_agent = create_react_agent(llm, [planner_tools.planner_tool], prompt)
planner_agent_executor = AgentExecutor(agent=planner_agent, tools=[planner_tools.planner_tool], verbose=True, handle_parsing_errors=True, max_iterations=3, early_stopping_method="force")

def call_planner_agent(state: AgentState):
    result = planner_agent_executor.invoke({"input": f"Lập kế hoạch cho yêu cầu: {state['input']}"})
    return {"messages": [f"Kế hoạch: {result['output']}"]}

# weatherapi_agent = create_react_agent(llm, [weather_tools.weatherapi_tool], prompt) 
# weatherapi_agent_executor = AgentExecutor(agent=weatherapi_agent, tools=[weather_tools.weatherapi_tool], verbose=True, handle_parsing_errors=True) 

# def call_weatherapi_agent(state: AgentState):
#     result = weatherapi_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
#     return {"messages": [f"Thời tiết tại: {result['output']}"]}

In [55]:
graph_builder = StateGraph(AgentState)

# Nodes
graph_builder.add_node("planner", call_planner_agent)
graph_builder.add_node("sentiment_analyzer", call_sentiment_agent)
graph_builder.add_node("poet", call_poet_agent)
graph_builder.add_node("weather_llm", call_weatherllm_agent)
graph_builder.add_node("travel_analyzer", call_travel_agent)
#graph_builder.add_node("weather_api", call_weatherapi_agent)

# Entry at planner
graph_builder.set_entry_point("planner")

# Edges
graph_builder.add_edge("planner", "sentiment_analyzer")
graph_builder.add_edge("sentiment_analyzer", "poet")
graph_builder.add_edge("poet", "weather_llm")
graph_builder.add_edge("weather_llm", "travel_analyzer")
#graph_builder.add_edge("weather_llm", "weather_api")
#graph_builder.add_edge("weather_api", END)
graph_builder.add_edge("travel_analyzer", END)


graph = graph_builder.compile()

input_test = "Hãy phân tích cảm xúc của đoạn text sau: 'Một ngày đẹp trời với bầu trời trong xanh và mặt trời tỏa nắng ấm áp.'"

final_state = graph.invoke({"input": input_test})
print("Kết quả cuối cùng:")
for message in final_state["messages"]:
    print("- ", message)

# Hiển thị lịch sử hội thoại đã được các tool lưu
print("\nLịch sử hội thoại (gần nhất):")
for msg in memory_manager.get_recent_history(user_id="default_user", n=10):
    print(f"{msg['timestamp']} | {msg['role']}: {msg['content']}")



> Entering new AgentExecutor chain...
Let's start by thinking about what we need to do to analyze the emotions in this text.

Thought: To plan for this task, I should define what emotions we are looking for and what aspects of the text we want to examine. We might consider the tone, vocabulary, and imagery used in the text.

Action: Plan
Action Input: Define emotions and text analysis aspectsPlan is not a valid tool, try one of [Planner].It seems I made a mistake!

Since I have access to the Planner tool, let me retry:

Action: Planner
Action Input: Define emotions and text analysis aspectsHere's the planner output:

**Subgoal Decomposition**

1. Identify key emotions to analyze
	* Happiness
	* Sadness
	* Anger
	* Fear
2. Determine relevant text features for each emotion
	* Sentiment (positive/negative)
	* Tone (formal/informal)
	* Grammar and syntax

**Action Steps**

1. Conduct literature review on emotions and text analysis
	* Review existing research on emotions and text analysis

ValueError: Got unsupported early_stopping_method `generate`

In [ ]:
# Session and memory inspection
user_id = "default_user"
session_id = memory_manager.get_session_id(user_id)
print(f"Current session: {session_id}")

print("\nRecent short-term history:")
for msg in memory_manager.get_recent_history(n=20, user_id=user_id):
    print(f"{msg['timestamp']} | {msg['role']}: {msg['content']}")

history = memory_manager.list_conversation_history(user_id=user_id)
last_input = history[-1]["content"] if history else "tổng quan"

print("\nContext summary (with long-term):")
summary = memory_manager.get_context_summary(user_id=user_id, include_long_term=True, current_input=last_input)
print(summary)

print("\nAll session_ids in long-term:")
print(memory_manager.list_sessions())


Current session: session_20250928_092221

Recent short-term history:
2025-09-28T15:22:47.459830 | user: Thời tiết tại Hà Nội ra sao, Tôi muốn lên kế hoạch du lịch sắp tới
2025-09-28T15:22:47.459830 | user: [Tool Input][SentimentAnalyzer] Phân tích cảm xúc của đoạn text sau: Thời tiết tại Hà Nội ra sao, Tôi muốn lên kế hoạch du lịch sắp tới

2025-09-28T15:22:47.459830 | tool:SentimentAnalyzer: Phân tích cảm xúc: Positive

Vì người dùng đang hỏi về thời tiết và lên kế hoạch du lịch, thể hiện mong muốn và hứng thú.
2025-09-28T15:22:47.459830 | user: [Tool Input][PoemWriter] Thời tiết tại Hà Nội ra sao (Hanoi's weather)
2025-09-28T15:22:47.459830 | tool:PoemWriter: Here is a short poem with 4 lines on the topic of Hanoi's weather:

Misty mornings dawn in Hanoi's sky
Soft rains fall, washing away the sigh
Golden sunsets paint the city bright
Perfect weather for a traveler's delight
2025-09-28T15:22:47.459830 | assistant: Kế hoạch: Agent stopped due to iteration limit or time limit.
Phân tíc

Session: session_20250928_092221
Recent conversation:
assistant: Kế hoạch: Agent stopped due to iteration limit or time limit.
Phân tích cảm xúc: Agent stopped due to iteration limit or time limit.
Bài thơ: Agent stopped due to iteration limit or time limit.
Thời tiết tại: Agent stopped due to iteration limit or time limit.
Phân tích lịch trình du lịch: Agent stopped due to iteration limit or time limit.
user: Hà Nội thật đẹp, hãy làm bài thơ về nơi này
assistant: Kế hoạch: Agent stopped due to iteration limit or time limit.
Phân tích cảm xúc: Agent stopped due to iteration limit or time limit.
Bài thơ: Agent stopped due to iteration limit or time limit.
Thời tiết tại: Agent stopped due to iteration limit or time limit.
Phân tích lịch trình du lịch: Agent stopped due to iteration limit or time limit.


Thông tin từ các cuộc trò chuyện trước:
1. assistant: Kế hoạch: Agent stopped due to iteration limit or time limit.
Phân tích cảm xúc: Agent stopped due to iteration limit or time limit.

In [ ]:
# Read long-term memory by session_id
# Lists available sessions, then reads and prints all messages for target session
from Source.ai.Multi_Agent.Source.Main.Memory.memory.long_term_memory import long_term_memory as ltm

# Show available session IDs in long-term store
available_sessions = memory_manager.list_sessions()
print("Available session_ids:")
print(available_sessions)

# Choose which session to read: default to current session
target_session_id = "session_20250928_092221"   # e.g., "session_20250927_164702"
print(f"\nReading long-term memory for session: {target_session_id}")

results = ltm.collection.get(
    where={"session_id": target_session_id}, include=["documents", "metadatas"]
) or {}

documents = results.get("documents") or []
metadatas = results.get("metadatas") or []

records = []
for doc, meta in zip(documents, metadatas):
    timestamp = (meta or {}).get("timestamp", "")
    role = (meta or {}).get("role", "system")
    content = doc
    if isinstance(doc, str) and ": " in doc:
        content = doc.split(": ", 1)[1]
    records.append((timestamp, role, content))

# Sort by timestamp ascending if timestamps are present
records.sort(key=lambda x: x[0])

print(f"Total messages: {len(records)}")
for ts, role, content in records:
    print(f"{ts} | {role}: {content}")



Available session_ids:
['session_20250925_130323', 'session_20250925_144950', 'session_20250925_150303', 'session_20250925_151001', 'session_20250927_162902', 'session_20250927_164507', 'session_20250928_092221']

Reading long-term memory for session: session_20250928_092221
Total messages: 83
2025-09-28T09:22:22.861058 | user: [Tool Input][SentimentAnalyzer] Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại.
2025-09-28T09:22:36.943530 | tool:SentimentAnalyzer: Based on the given text, I would classify the sentiment as "Neutral". The text only provides factual information about the current weather in Ho Chi Minh City, without expressing any emotional tone or opinion.
2025-09-28T09:23:04.743341 | user: [Tool Input][SentimentAnalyzer] Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại.
2025-09-28T09:23:20.098218 | tool:SentimentAnalyzer: Based on the given text, I would classify the sentiment as "Neutral". The text only provides factual information about the current weather in Ho 

In [ ]:
# Chat loop with session resume (+ /plan command)
user_id = "default_user"

# Chọn session để tiếp tục
available = memory_manager.list_sessions()
print("Các session có sẵn:")
print(available)
choice = input("Nhập session_id để tiếp tục, 'new' để tạo mới, Enter để dùng session hiện tại: ").strip()

if choice.lower() == "new":
    new_id = memory_manager.start_new_session(user_id=user_id, clear_history=True, keep_preferences=True)
    print(f"Bắt đầu session mới: {new_id}")
elif choice:
    try:
        loaded = memory_manager.resume_session(choice, user_id=user_id, replay_last_n=20)
        print(f"Khôi phục {loaded} tin nhắn cho session: {choice}")
    except Exception as e:
        print(f"Không thể khôi phục session: {e}")

current = memory_manager.get_session_id(user_id)
print(f"Đang dùng session: {current}")
print("Gõ 'q' để thoát. Hỗ trợ lệnh: /plan <nội dung>")

last_assistant_reply = ""

while True:
    user_text = input("Bạn: ").strip()
    if user_text.lower() in ("q", "quit", "exit"):
        print("Kết quả cuối cùng:")
        if last_assistant_reply:
            print(last_assistant_reply)
        else:
            print("Chưa có kết quả trong phiên.")
        print("\nLịch sử hội thoại (gần nhất):")
        for msg in memory_manager.get_recent_history(user_id=user_id, n=5):
            print(f"{msg['timestamp']} | {msg['role']}: {msg['content']}")
        print("Kết thúc.")
        break

    # Nhánh lệnh /plan: chỉ gọi Planner, không chạy toàn chuỗi
    if user_text.startswith("/plan "):
        plan_input = user_text[6:].strip()
        memory_manager.add_message(role="user", content=f"[Command]/plan {plan_input}", user_id=user_id, save_to_long_term=True)
        try:
            plan_result = planner_agent_executor.invoke({"input": plan_input})
            assistant_reply = plan_result.get("output", "")
        except Exception as e:
            assistant_reply = f"Lỗi khi gọi Planner: {e}"
        memory_manager.add_message(role="assistant", content=assistant_reply, user_id=user_id, save_to_long_term=True)
        print(f"Assistant:\n{assistant_reply}")
        last_assistant_reply = assistant_reply
        continue

    # Mặc định: Lưu message của người dùng và gọi toàn graph
    memory_manager.add_message(role="user", content=user_text, user_id=user_id, save_to_long_term=True)

    try:
        result_state = graph.invoke({"input": user_text})
        outputs = result_state.get("messages", [])
        assistant_reply = "\n".join(outputs) if isinstance(outputs, list) else str(outputs)
    except Exception as e:
        assistant_reply = f"Lỗi khi gọi agent: {e}"

    # Làm sạch thông điệp lặp/dừng
    clean_reply = "\n".join([line for line in assistant_reply.splitlines() if "Agent stopped due to iteration limit or time limit" not in line])

    memory_manager.add_message(role="assistant", content=clean_reply, user_id=user_id, save_to_long_term=True)
    print(f"Assistant:\n{clean_reply}")
    last_assistant_reply = clean_reply



Các session có sẵn:
['session_20250925_130323', 'session_20250925_144950', 'session_20250925_150303', 'session_20250925_151001', 'session_20250927_162902', 'session_20250927_164507', 'session_20250928_092221']
Khôi phục 20 tin nhắn cho session: session_20250928_092221
Đang dùng session: session_20250928_092221
Gõ 'q' để thoát. Hỗ trợ lệnh: /plan <nội dung>


> Entering new AgentExecutor chain...
Let's start planning a trip to Hanoi, Vietnam.

Thought: To create a structured plan for my trip to Hanoi, I need to break it down into subgoals, steps, assumptions/risks, self-critique, and reflection. Let's start by defining the purpose of my trip and setting some goals.

Action: Planner(plan_input="Define Trip Purpose")
Action Input: NonePlanner(plan_input="Define Trip Purpose") is not a valid tool, try one of [Planner].I'm excited to help you plan your trip to Hanoi!

Thought: I think there might be some confusion. It seems that the action I was supposed to take with the input "Define Trip 